In [1]:
import pandas as pd
import numpy as np

In [2]:
import allel # ref - https://scikit-allel.readthedocs.io/en/stable/io.html#variant-call-format-vcf

In [3]:
vcf_folder = "../dataset/"
vcf_file_freebayes = "syn5-freebayes.vcf.gz"
vcf_file_mutect = "syn5-mutect2.vcf.gz"
vcf_file_vardict = "syn5-vardict.vcf.gz"
vcf_file_varscan = "syn5-varscan.vcf.gz"

In [4]:
df_freebayes = allel.vcf_to_dataframe(vcf_folder + vcf_file_freebayes)

In [5]:
df_freebayes.head()

,CHROM,POS,ID,REF,ALT_1,ALT_2,ALT_3,QUAL,FILTER_PASS
0,1,10057,.,A,AC,NaN,NaN,56.900002,False
1,1,10248,rs148908337,A,T,NaN,NaN,178.899994,False
2,1,10255,rs1015259029,A,T,NaN,NaN,193.399994,False
3,1,10260,.,C,CT,NaN,NaN,61.599998,False
4,1,10427,.,A,AC,NaN,NaN,65.199997,False


In [6]:
df_mutect = allel.vcf_to_dataframe(vcf_folder + vcf_file_mutect)
df_mutect.head()

,CHROM,POS,ID,REF,ALT_1,ALT_2,ALT_3,QUAL,FILTER_PASS
0,1,674456,rs77806715,G,A,NaN,NaN,NaN,False
1,1,726156,rs796779711,T,A,NaN,NaN,NaN,False
2,1,726173,.,G,A,NaN,NaN,NaN,False
3,1,726183,rs1000851233,C,A,NaN,NaN,NaN,False
4,1,726185,rs111203408,T,A,NaN,NaN,NaN,False


In [7]:
df_vardict = allel.vcf_to_dataframe(vcf_folder + vcf_file_vardict)
df_vardict.head()

,CHROM,POS,ID,REF,ALT_1,ALT_2,ALT_3,QUAL,FILTER_PASS
0,1,10248,rs148908337,A,T,NaN,NaN,42.0,False
1,1,10255,rs1015259029,A,T,NaN,NaN,48.0,False
2,1,10404,.,CCCTAACCCTAACCCTAACCCTAA,C,NaN,NaN,35.0,False
3,1,10439,rs112766696,AC,A,NaN,NaN,63.0,False
4,1,10583,rs58108140,G,A,NaN,NaN,67.0,False


In [8]:
df_varscan = allel.vcf_to_dataframe(vcf_folder + vcf_file_varscan)
df_varscan.head()

,CHROM,POS,ID,REF,ALT_1,ALT_2,ALT_3,QUAL,FILTER_PASS
0,1,10150,rs371194064,C,T,NaN,NaN,0.0,False
1,1,10352,rs555500075;rs145072688,T,TA,NaN,NaN,0.0,False
2,1,10427,.,A,AC,NaN,NaN,0.0,False
3,1,10583,rs58108140,G,A,NaN,NaN,0.0,False
4,1,14464,rs546169444,A,T,NaN,NaN,0.0,False


In [9]:
print(f"Real freebayes shape = {df_freebayes.shape}")
print(f"Real Mutect shape = {df_mutect.shape}")
print(f"Real Vardict shape = {df_vardict.shape}")
print(f"Real Varscan shape = {df_varscan.shape}")

Real freebayes shape = (4621247, 9)
Real Mutect shape = (116115, 9)
Real Vardict shape = (4502129, 9)
Real Varscan shape = (4368655, 9)


In [10]:
m1 = pd.merge(df_freebayes, df_mutect, on = ["CHROM", "POS"], how="outer", suffixes = ("_freebayes", "_mutect"))

In [11]:
m1.shape

(4682646, 16)

In [12]:
m2 = pd.merge(m1, df_vardict, on = ["CHROM", "POS"], how="outer")
m2.rename(columns={"ID": "ID_vardict", "REF": "REF_vardict", "ALT_1": "ALT_1_vardict", "ALT_2": "ALT_2_vardict", \
                  "ALT_3": "ALT_3_vardict", "QUAL": "QUAL_vardict", "FILTER_PASS": "FILTER_PASS_vardict"}, inplace=True)

In [13]:
m2.head()

,CHROM,POS,ID_freebayes,REF_freebayes,ALT_1_freebayes,ALT_2_freebayes,ALT_3_freebayes,QUAL_freebayes,FILTER_PASS_freebayes,ID_mutect,...,ALT_3_mutect,QUAL_mutect,FILTER_PASS_mutect,ID_vardict,REF_vardict,ALT_1_vardict,ALT_2_vardict,ALT_3_vardict,QUAL_vardict,FILTER_PASS_vardict
0,1,10057,.,A,AC,NaN,NaN,56.900002,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10248,rs148908337,A,T,NaN,NaN,178.899994,False,NaN,...,NaN,NaN,NaN,rs148908337,A,T,NaN,NaN,42.0,False
2,1,10255,rs1015259029,A,T,NaN,NaN,193.399994,False,NaN,...,NaN,NaN,NaN,rs1015259029,A,T,NaN,NaN,48.0,False
3,1,10260,.,C,CT,NaN,NaN,61.599998,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10427,.,A,AC,NaN,NaN,65.199997,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
m2.shape

(4867809, 23)

In [15]:
m3 = pd.merge(m2, df_varscan, on = ["CHROM", "POS"], how="outer")
m3.rename(columns={"ID": "ID_varscan", "REF": "REF_varscan", "ALT_1": "ALT_1_varscan", "ALT_2": "ALT_2_varscan", \
                  "ALT_3": "ALT_3_varscan", "QUAL": "QUAL_varscan", "FILTER_PASS": "FILTER_PASS_varscan"}, inplace=True)
m3.shape

(4977710, 30)

In [16]:
m3.head()

,CHROM,POS,ID_freebayes,REF_freebayes,ALT_1_freebayes,ALT_2_freebayes,ALT_3_freebayes,QUAL_freebayes,FILTER_PASS_freebayes,ID_mutect,...,ALT_3_vardict,QUAL_vardict,FILTER_PASS_vardict,ID_varscan,REF_varscan,ALT_1_varscan,ALT_2_varscan,ALT_3_varscan,QUAL_varscan,FILTER_PASS_varscan
0,1,10057,.,A,AC,NaN,NaN,56.900002,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10248,rs148908337,A,T,NaN,NaN,178.899994,False,NaN,...,NaN,42.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,10255,rs1015259029,A,T,NaN,NaN,193.399994,False,NaN,...,NaN,48.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,10260,.,C,CT,NaN,NaN,61.599998,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10427,.,A,AC,NaN,NaN,65.199997,False,NaN,...,NaN,NaN,NaN,.,A,AC,NaN,NaN,0.0,False


In [17]:
m3.head()[["CHROM", "POS", "FILTER_PASS_freebayes", "FILTER_PASS_mutect", "FILTER_PASS_vardict", "FILTER_PASS_varscan"]]

,CHROM,POS,FILTER_PASS_freebayes,FILTER_PASS_mutect,FILTER_PASS_vardict,FILTER_PASS_varscan
0,1,10057,False,NaN,NaN,NaN
1,1,10248,False,NaN,False,NaN
2,1,10255,False,NaN,False,NaN
3,1,10260,False,NaN,NaN,NaN
4,1,10427,False,NaN,NaN,False


In [18]:
df_merged = m3

In [19]:
# add third category instead of False?
df_merged["FILTER_PASS_freebayes"].fillna(False, inplace=True)
df_merged["FILTER_PASS_vardict"].fillna(False, inplace=True)
df_merged["FILTER_PASS_mutect"].fillna(False, inplace=True)
df_merged["FILTER_PASS_varscan"].fillna(False, inplace=True)
df_merged.head()[["CHROM", "POS", "FILTER_PASS_freebayes", "FILTER_PASS_mutect", "FILTER_PASS_vardict", "FILTER_PASS_varscan"]]

,CHROM,POS,FILTER_PASS_freebayes,FILTER_PASS_mutect,FILTER_PASS_vardict,FILTER_PASS_varscan
0,1,10057,False,False,False,False
1,1,10248,False,False,False,False
2,1,10255,False,False,False,False
3,1,10260,False,False,False,False
4,1,10427,False,False,False,False


In [20]:
truth = pd.read_csv(vcf_folder + "syn5_truth.bed", sep = "\t", header = None)
truth.columns = ["CHROM", "POS_START", "POS_END"]
truth.head()

,CHROM,POS_START,POS_END
0,1,752211,752211
1,1,824652,824652
2,1,917467,917467
3,1,1016648,1016648
4,1,1038806,1038806


In [21]:
(truth.POS_START == truth.POS_END).sum()

45383

In [22]:
truth.shape

(45383, 3)

In [23]:
truth.merge(df_merged, left_on = ["CHROM", "POS_START"], right_on = ["CHROM", "POS"])
# this is smaller than the actual truth file indicating the all 4 variant callers missed to classify some true variants

,CHROM,POS_START,POS_END,POS,ID_freebayes,REF_freebayes,ALT_1_freebayes,ALT_2_freebayes,ALT_3_freebayes,QUAL_freebayes,...,ALT_3_vardict,QUAL_vardict,FILTER_PASS_vardict,ID_varscan,REF_varscan,ALT_1_varscan,ALT_2_varscan,ALT_3_varscan,QUAL_varscan,FILTER_PASS_varscan
0,1,752211,752211,752211,.,G,C,NaN,NaN,11.400000,...,NaN,70.0,True,.,G,C,NaN,NaN,0.0,True
1,1,824652,824652,824652,.,T,A,NaN,NaN,321.100006,...,NaN,131.0,True,.,T,A,NaN,NaN,0.0,True
2,1,917467,917467,917467,.,G,T,NaN,NaN,32.099998,...,NaN,35.0,False,.,G,T,NaN,NaN,0.0,True
3,1,1016648,1016648,1016648,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,34.0,False,NaN,NaN,NaN,NaN,NaN,NaN,False
4,1,1038806,1038806,1038806,.,G,T,NaN,NaN,245.500000,...,NaN,120.0,True,.,G,T,NaN,NaN,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45089,Y,28483053,28483053,28483053,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,33.0,False,.,A,C,NaN,NaN,0.0,True
45090,Y,28656099,28656099,28656099,.,G,C,NaN,NaN,116.400002,...,NaN,108.0,True,.,G,C,NaN,NaN,0.0,False
45091,Y,28713794,28713794,28713794,.,A,T,NaN,NaN,143.699997,...,NaN,119.0,True,.,A,T,NaN,NaN,0.0,False
45092,Y,28762270,28762270,28762270,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,81.0,True,.,A,C,NaN,NaN,0.0,False


In [24]:
df_merged_with_truth = pd.merge(truth, df_merged, left_on=["CHROM", "POS_START"], right_on = ["CHROM", "POS"], how="outer")
df_merged_with_truth.head()

,CHROM,POS_START,POS_END,POS,ID_freebayes,REF_freebayes,ALT_1_freebayes,ALT_2_freebayes,ALT_3_freebayes,QUAL_freebayes,...,ALT_3_vardict,QUAL_vardict,FILTER_PASS_vardict,ID_varscan,REF_varscan,ALT_1_varscan,ALT_2_varscan,ALT_3_varscan,QUAL_varscan,FILTER_PASS_varscan
0,1,752211.0,752211.0,752211.0,.,G,C,NaN,NaN,11.400000,...,NaN,70.0,True,.,G,C,NaN,NaN,0.0,True
1,1,824652.0,824652.0,824652.0,.,T,A,NaN,NaN,321.100006,...,NaN,131.0,True,.,T,A,NaN,NaN,0.0,True
2,1,917467.0,917467.0,917467.0,.,G,T,NaN,NaN,32.099998,...,NaN,35.0,False,.,G,T,NaN,NaN,0.0,True
3,1,1016648.0,1016648.0,1016648.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,34.0,False,NaN,NaN,NaN,NaN,NaN,NaN,False
4,1,1038806.0,1038806.0,1038806.0,.,G,T,NaN,NaN,245.500000,...,NaN,120.0,True,.,G,T,NaN,NaN,0.0,True


In [25]:
df_merged_with_truth.shape

(4977999, 32)

In [26]:
def set_true_label(row):
    if str(row) == "nan":
        return False
    else:
        return True

In [27]:
df_merged_with_truth["truth"] = df_merged_with_truth["POS_START"].apply(lambda row:set_true_label(row))

In [28]:
extras_in_truth = df_merged_with_truth[df_merged_with_truth.POS.isna()].index
extras_in_truth

Int64Index([   10,   121,   293,   374,   467,   519,   858,  1116,  1228,
             1281,
            ...
            42163, 42392, 42572, 42694, 43070, 43090, 43421, 44284, 44356,
            45342],
           dtype='int64', length=289)

In [29]:
df_merged_with_truth.drop(extras_in_truth, inplace=True)

In [30]:
df_merged_with_truth.isna().sum()

CHROM                          0
POS_START                4932616
POS_END                  4932616
POS                            0
ID_freebayes              356463
REF_freebayes             356463
ALT_1_freebayes           356463
ALT_2_freebayes          4943335
ALT_3_freebayes          4977705
QUAL_freebayes            356464
FILTER_PASS_freebayes          0
ID_mutect                4861593
REF_mutect               4861593
ALT_1_mutect             4861593
ALT_2_mutect             4977710
ALT_3_mutect             4977710
QUAL_mutect              4977710
FILTER_PASS_mutect             0
ID_vardict                465937
REF_vardict               465937
ALT_1_vardict             465937
ALT_2_vardict            4977710
ALT_3_vardict            4977710
QUAL_vardict              466035
FILTER_PASS_vardict            0
ID_varscan                600711
REF_varscan               600711
ALT_1_varscan             600711
ALT_2_varscan            4977710
ALT_3_varscan            4977710
QUAL_varsc

In [31]:
df_merged_with_truth["QUAL_freebayes"].fillna(df_merged_with_truth["QUAL_freebayes"].mean(), inplace=True)
df_merged_with_truth["QUAL_vardict"].fillna(df_merged_with_truth["QUAL_vardict"].mean(), inplace=True)

In [32]:
df_merged_with_truth["QUAL_mutect"].isna().sum()

4977710

In [33]:
df_merged_with_truth[["QUAL_freebayes", "QUAL_vardict"]].corr()

,QUAL_freebayes,QUAL_vardict
QUAL_freebayes,1.000000,0.501959
QUAL_vardict,0.501959,1.000000


#### Confusion matrix comparing against truth

In [34]:
from sklearn.metrics import confusion_matrix

In [35]:
tn, fp, fn, tp = confusion_matrix(list(df_merged_with_truth.truth), list(df_merged_with_truth.FILTER_PASS_freebayes)).ravel()
tn, fp, fn, tp

(4921903, 10713, 2853, 42241)

In [36]:
# least false positives
tn, fp, fn, tp = confusion_matrix(list(df_merged_with_truth.truth), list(df_merged_with_truth.FILTER_PASS_mutect)).ravel()
tn, fp, fn, tp

(4915754, 16862, 1910, 43184)

In [37]:
tn, fp, fn, tp = confusion_matrix(list(df_merged_with_truth.truth), list(df_merged_with_truth.FILTER_PASS_vardict)).ravel()
tn, fp, fn, tp

(4917165, 15451, 1908, 43186)

In [38]:
tn, fp, fn, tp = confusion_matrix(list(df_merged_with_truth.truth), list(df_merged_with_truth.FILTER_PASS_varscan)).ravel()
tn, fp, fn, tp

(4924179, 8437, 19590, 25504)

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_merged_with_truth[["FILTER_PASS_mutect", "FILTER_PASS_freebayes", "FILTER_PASS_vardict", "FILTER_PASS_varscan"]], df_merged_with_truth["truth"], test_size = 0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3982168, 4)
(995542, 4)
(3982168,)
(995542,)


In [40]:
from pgmpy.models import BayesianNetwork

In [41]:
bn = BayesianNetwork(
[
#     ("CHROM", "truth"),
#     ("POS", "truth"),
#     ("QUAL_freebayes", "truth"),
#     ("QUAL_vardict", "truth"),
    ("FILTER_PASS_mutect", "truth"),
    ("FILTER_PASS_freebayes", "truth"),
    ("FILTER_PASS_vardict", "truth"),
    ("FILTER_PASS_varscan", "truth"),
])

In [42]:
from pgmpy.estimators import MaximumLikelihoodEstimator

In [43]:
train_df = pd.DataFrame(X_train, columns = ["FILTER_PASS_mutect","FILTER_PASS_freebayes", "FILTER_PASS_vardict", "FILTER_PASS_varscan"])
train_df["truth"] = y_train
print(train_df.shape)
train_df.head()

(3982168, 5)


,FILTER_PASS_mutect,FILTER_PASS_freebayes,FILTER_PASS_vardict,FILTER_PASS_varscan,truth
2905009,False,False,False,False,False
1930390,False,False,False,False,False
4867458,False,False,False,False,False
2998827,False,False,False,False,False
3111100,False,False,False,False,False


In [44]:
bn.fit(
    data=train_df,
    estimator=MaximumLikelihoodEstimator
)

In [45]:
print(bn.get_cpds("truth"))

+-----------------------+-----+-----------------------------+
| FILTER_PASS_freebayes | ... | FILTER_PASS_freebayes(True) |
+-----------------------+-----+-----------------------------+
| FILTER_PASS_mutect    | ... | FILTER_PASS_mutect(True)    |
+-----------------------+-----+-----------------------------+
| FILTER_PASS_vardict   | ... | FILTER_PASS_vardict(True)   |
+-----------------------+-----+-----------------------------+
| FILTER_PASS_varscan   | ... | FILTER_PASS_varscan(True)   |
+-----------------------+-----+-----------------------------+
| truth(False)          | ... | 0.22561307901907357         |
+-----------------------+-----+-----------------------------+
| truth(True)           | ... | 0.7743869209809264          |
+-----------------------+-----+-----------------------------+


In [46]:
test_df = pd.DataFrame(X_test, columns = ["FILTER_PASS_mutect","FILTER_PASS_freebayes", "FILTER_PASS_vardict", "FILTER_PASS_varscan"])
test_df["truth"] = y_test
print(test_df.shape)
test_df.head()

(995542, 5)


,FILTER_PASS_mutect,FILTER_PASS_freebayes,FILTER_PASS_vardict,FILTER_PASS_varscan,truth
2715787,False,False,False,False,False
1232643,False,False,False,False,False
2025889,False,False,False,False,False
2665164,False,False,False,False,False
1041334,False,False,False,False,False


In [47]:
from pgmpy.inference import VariableElimination
from ipywidgets import FloatProgress

In [48]:
bn_infer = VariableElimination(bn)

In [49]:
from IPython.display import clear_output

In [50]:
def get_preds(row):
    prob = bn_infer.query(variables=["truth"], evidence={"FILTER_PASS_freebayes": row["FILTER_PASS_freebayes"], \
                                             "FILTER_PASS_mutect": row["FILTER_PASS_mutect"], \
                                             "FILTER_PASS_vardict": row["FILTER_PASS_vardict"], \
                                             "FILTER_PASS_varscan": row["FILTER_PASS_varscan"]}, show_progress=False)
#     clear_output(wait=True)
    if (prob.values[1]) > 0.5:
        return True
    else:
        return False

In [51]:
test_df["preds"] = test_df.apply(get_preds, axis = 1)

#### Metric evaluation

In [52]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [53]:
from sklearn.metrics import classification_report

In [54]:
print(classification_report(test_df["truth"], test_df["preds"]))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00    986620
        True       0.79      0.95      0.86      8922

    accuracy                           1.00    995542
   macro avg       0.89      0.98      0.93    995542
weighted avg       1.00      1.00      1.00    995542

